MSc: learning a Gaussian with tensorflow

In [1]:
import tensorflow as tf
import numpy as np
import scipy.stats
import scipy.io
import scipy.sparse
from scipy.io import loadmat
import pandas as pd
import tensorflow_probability as tfp
tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers
from PIL import Image
import matplotlib.pyplot as plt

We load the iris data set.

In [2]:
from sklearn.datasets import load_iris
data = load_iris()['data']

In [3]:
x = ((data - np.mean(data,0))/np.std(data,0)).astype(np.float32)
n = x.shape[0] # number of observations
p = x.shape[1] # number of features

Learn a Gaussian model $p(x) = \mathcal{N}(x|\mu,\Sigma)$ using Tensorflow!

In [4]:
mu = tf.Variable(tf.random.normal([p]), trainable=True)
L_undiag = tf.Variable(tfp.math.fill_triangular(tf.random.normal([int(p*(p+1)/2)]))) 

In [5]:
L = tf.linalg.set_diag(L_undiag,tf.exp(tf.linalg.diag_part(L_undiag))) # L corresponds to the Cholesky decomposition

In [6]:
Sigma = tf.matmul(L,L,transpose_b= True) # L*L^T

We can check that $\Sigma$ is indeed a positive definite matrix, by checking that it's indeed symmetric with nonnegative eigenvalues.

In [7]:
print(Sigma.numpy()) 
print(np.linalg.eigvals(Sigma.numpy()))

[[ 3.3217838  2.7583165 -2.073172   1.3955039]
 [ 2.7583165 17.793037   6.5375595  0.6485721]
 [-2.073172   6.5375595  6.230428   0.3784474]
 [ 1.3955039  0.6485721  0.3784474  5.350596 ]]
[2.0950649e+01 1.7065268e-02 6.7815847e+00 4.9465461e+00]


Let's start by computing the likelihood of a single data point.

In [8]:
gaussian = tfd.MultivariateNormalFullCovariance(loc = mu, covariance_matrix = Sigma) 

Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.


In [9]:
gaussian.log_prob(x[:1,:])

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-217.53217], dtype=float32)>

We'll do a custom training loop for our unknown mean. We first define our loss function.

In [10]:
def log_likelihood(x):
  L = tf.linalg.set_diag(L_undiag,tf.exp(tf.linalg.diag_part(L_undiag)))
  Sigma = tf.matmul(L,L,transpose_b= True) 
  gaussian = tfd.MultivariateNormalFullCovariance(loc = mu, covariance_matrix = Sigma)
  return(tf.reduce_mean(gaussian.log_prob(x)))

Then, we define the paramaters we want to optimise, and we define a gradient step.

In [11]:
params = [mu] + [L_undiag]
optimizer = tfk.optimizers.Adam(learning_rate = 0.01)

In [12]:
def train_step(data): # data is a batch of data points
  with tf.GradientTape() as tape:
    loss = - log_likelihood(data)
  gradients = tape.gradient(loss, params) # here we compute gradients
  optimizer.apply_gradients(zip(gradients,params))

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices((x)).shuffle(n).batch(batch_size = 16)

In [14]:
for epoch in range(200):
  for data_batch in train_dataset:
    train_step(data_batch)
  if (epoch % 10) == 1:
    print(log_likelihood(x))

tf.Tensor(-12.608184, shape=(), dtype=float32)
tf.Tensor(-8.395755, shape=(), dtype=float32)
tf.Tensor(-7.686512, shape=(), dtype=float32)
tf.Tensor(-7.282068, shape=(), dtype=float32)
tf.Tensor(-6.9156876, shape=(), dtype=float32)
tf.Tensor(-6.528509, shape=(), dtype=float32)
tf.Tensor(-6.107011, shape=(), dtype=float32)
tf.Tensor(-5.657769, shape=(), dtype=float32)
tf.Tensor(-5.118271, shape=(), dtype=float32)
tf.Tensor(-4.3964, shape=(), dtype=float32)
tf.Tensor(-4.006534, shape=(), dtype=float32)
tf.Tensor(-3.718487, shape=(), dtype=float32)
tf.Tensor(-3.4759986, shape=(), dtype=float32)
tf.Tensor(-3.3386252, shape=(), dtype=float32)
tf.Tensor(-3.305196, shape=(), dtype=float32)
tf.Tensor(-3.2854435, shape=(), dtype=float32)
tf.Tensor(-3.2791643, shape=(), dtype=float32)
tf.Tensor(-3.2717423, shape=(), dtype=float32)
tf.Tensor(-3.2697456, shape=(), dtype=float32)
tf.Tensor(-3.2697082, shape=(), dtype=float32)


In [15]:
mu

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0.00570357, 0.00027669, 0.01256279, 0.00723899], dtype=float32)>

In [16]:
  L = tf.linalg.set_diag(L_undiag,tf.exp(tf.linalg.diag_part(L_undiag)))
  Sigma = tf.matmul(L,L,transpose_b= True) # L*L^T

In [17]:
Sigma

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[ 1.0315608 , -0.13662878,  0.9013501 ,  0.8399476 ],
       [-0.13662878,  0.9979911 , -0.44282168, -0.3924243 ],
       [ 0.9013501 , -0.44282168,  1.0279208 ,  0.9879363 ],
       [ 0.8399476 , -0.3924243 ,  0.9879363 ,  1.0253401 ]],
      dtype=float32)>

In [18]:
np.cov(x, rowvar = False)

array([[ 1.00671141, -0.11835884,  0.87760447,  0.82343068],
       [-0.11835884,  1.0067114 , -0.43131554, -0.36858316],
       [ 0.87760447, -0.43131554,  1.0067114 ,  0.96932763],
       [ 0.82343068, -0.36858316,  0.96932763,  1.00671144]])